# Tutorial 1: Introduction to Gridap.jl - From Weak Form to Assembly

Welcome to the first tutorial in the CMOE series on computational methods for multiphysics problems! In this tutorial, you will learn the fundamentals of using **Gridap.jl**, a powerful finite element library in Julia, to solve partial differential equations (PDEs).

> **📘 About this tutorial**: The intention of this tutorial is to go in depth in the integration/assembly process. The set of tutorials provided in this repository are complementary to the [official Gridap Tutorials](https://gridap.github.io/Tutorials/dev/). We recommend the reader to have a look at those first.

> **⚠️ Note**: The text in this tutorial has been generated by LLMs. If you find any inconsistencies, please report them through an issue in the repository.

## Table of Contents

1. [Setting Up the Julia Environment](#1-setting-up-the-julia-environment)
2. [Creating a Computational Domain with Cartesian Meshes](#2-creating-a-computational-domain-with-cartesian-meshes)
3. [Defining Reference Finite Elements and Finite Element Spaces](#3-defining-reference-finite-elements-and-finite-element-spaces)
4. [Defining Triangulation and Integration Measures](#4-defining-triangulation-and-integration-measures)
5. [Weak Form Definition and Solution](#5-weak-form-definition-and-solution)
6. [Inspecting the Assembled System Matrices](#6-inspecting-the-assembled-system-matrices)
7. [Understanding Basis Functions: Reference vs Physical Domain](#7-understanding-basis-functions-reference-vs-physical-domain)
8. [Finite Element Functions: Free vs Fixed Degrees of Freedom](#8-finite-element-functions-free-vs-fixed-degrees-of-freedom)
9. [Evaluating Weak Forms with Basis Functions](#9-evaluating-weak-forms-with-basis-functions)

## Learning Objectives

By the end of this tutorial, you will be able to:
- Create computational domains using Cartesian meshes
- Define finite element spaces and reference elements
- Formulate weak forms of PDEs
- Assemble and solve linear systems
- Visualize solutions using VTK output

We will go deep on what happens under the hood within the assembly process in the Gridap.jl library. 

## Problem Statement

We will solve the classic **Poisson equation** on a 2D domain:

$$-\Delta u = f \quad \text{in } \Omega$$

with appropriate boundary conditions. This fundamental problem appears in many engineering applications including heat transfer, electrostatics, and potential flow.

## Prerequisites

- Basic understanding of finite element methods
- Familiarity with Julia programming language
- Knowledge of partial differential equations

Let's get started!

## 1. Setting Up the Julia Environment

Before we start working with Gridap.jl, we need to properly set up our Julia environment. This involves two critical steps:

### 1.1 Activating the Project Environment 
```julia
Pkg.activate("..")
```

When you activate a project environment, Julia switches from the global package registry to a **local project-specific environment**. This means:

- **Reproducibility**: The project uses the exact package versions specified in `Project.toml` and `Manifest.toml` files
- **Isolation**: Changes to packages in this project won't affect other Julia projects on your system
- **Consistency**: Everyone working on this tutorial will use the same package versions, avoiding "works on my machine" issues

The `".."` argument tells Julia to activate the environment in the parent directory.

### 1.2 Instantiating the Environment
```julia
Pkg.instantiate()
```

The `instantiate()` command ensures all required packages are installed with their correct versions:

- **Downloads missing packages**: If any packages specified in `Project.toml` are not installed, they will be downloaded
- **Resolves dependencies**: Ensures all package dependencies are compatible and available
- **Builds packages**: Compiles any binary dependencies that packages might need

Think of this as similar to `npm install` (Node.js), `pip install -r requirements.txt` (Python), or `maven install` (Java).

**Note**: You only need to run `instantiate()` once when you first clone or download this tutorial. After that, simply activating the environment is sufficient, unless the package dependencies change.

In [16]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()

  Activating project at `~/Progs/git_repos/CMOE/CMOE-Tutorials`


## 2. Creating a Computational Domain with Cartesian Meshes

In finite element analysis, the first step is to discretize our computational domain into a mesh. Gridap.jl provides the `CartesianDiscreteModel` function to create structured Cartesian meshes efficiently.

### 2.1 Understanding the Domain Setup

Let's break down the components:

1. **Domain Definition**: `domain = (0, L, 0, L)`
   - Defines a square domain from $(0,0)$ to $(L,L)$ in 2D space
   - The tuple format is `(x_min, x_max, y_min, y_max)`
   - For our example, with `L = 2.0`, we create a $2.0 \times 2.0$ square

2. **Mesh Partition**: `partition = (n, n)`
   - Specifies how many cells to create in each direction
   - With `n = 2`, we divide the domain into $2 \times 2 = 4$ square cells
   - Each cell will have dimensions $\frac{L}{n} \times \frac{L}{n} = 1.0 \times 1.0$

3. **Creating the Model**: `model = CartesianDiscreteModel(domain, partition)`
   - Generates the discrete model containing:
     - **Nodes (vertices)**: $(n+1) \times (n+1) = 9$ nodes in our case
     - **Cells (elements)**: $n \times n = 4$ quadrilateral elements
     - **Boundary tags**: Automatically labels domain boundaries for applying boundary conditions

### 2.2 Visualization with VTK

The `writevtk(model, "model")` function exports the mesh to VTK format:
- Creates files `model_0.vtu`, `model_1.vtu`, `model_2.vtu` (for different topological dimensions)
- These files can be visualized using [ParaView](https://www.paraview.org/) or other VTK-compatible software
- Useful for inspecting the mesh quality and structure before solving

> **📘 Note**: Cartesian meshes are ideal for rectangular domains and provide uniform element sizes, which is excellent for learning and debugging. For more complex geometries, Gridap also supports unstructured meshes imported from external mesh generators (like GMSH).

In [17]:
using Gridap
L = 2.0
n = 2
domain = (0,L,0,L)
partition = (n,n)
model = CartesianDiscreteModel(domain,partition)
writevtk(model,"model")

3-element Vector{Vector{String}}:
 ["model_0.vtu"]
 ["model_1.vtu"]
 ["model_2.vtu"]

## 3. Defining Reference Finite Elements and Finite Element Spaces

After creating the mesh, we need to define the **finite element spaces** where our solution will live. This involves two key concepts: the reference finite element and the function spaces built from it.

### 3.1 Reference Finite Element

```julia
reffe = ReferenceFE(lagrangian, Float64, 1)
```

The `ReferenceFE` defines the type of finite element basis functions we'll use:

- **`lagrangian`**: Specifies Lagrangian (nodal) interpolation
  - Basis functions have the property $\phi_i(x_j) = \delta_{ij}$ (Kronecker delta)
  - Values at nodes directly correspond to degrees of freedom
  
- **`Float64`**: The data type for storing values (double precision floating point)

- **`1`**: The polynomial order of the basis functions
  - Order 1 = linear elements (straight edges, planar faces)
  - For 2D quadrilaterals, this gives bilinear basis functions
  - Each element will have 4 nodes (one at each corner)

### 3.2 Test Function Space (V)

```julia
V = FESpace(model, reffe; conformity=:H1, dirichlet_tags="boundary")
```

The test space `V` is where we search for our solution. Key parameters:

- **`model`**: Links the space to our computational mesh
- **`reffe`**: Uses the reference element defined above
- **`conformity=:H1`**: Ensures functions are continuous across element boundaries
  - $H^1$ conformity means $u \in H^1(\Omega)$: functions with square-integrable derivatives
  - Essential for problems involving gradients (like our Poisson equation)
- **`dirichlet_tags="boundary"`**: Identifies where to apply Dirichlet boundary conditions
  - Tags all nodes on the domain boundary
  - These will be treated as constrained degrees of freedom

### 3.3 Trial Function Space (U)

```julia
U = TrialFESpace(V, x -> x[1])
```

The trial space `U` is constructed from the test space `V` with prescribed boundary values:

- **`V`**: Inherits structure from the test space
- **`x -> x[1]`**: Boundary condition function
  - For each point `x` on the boundary, sets $u = x_1$ (the x-coordinate)
  - This creates a non-homogeneous Dirichlet condition: $u(x_1, x_2) = x_1$ on $\partial\Omega$
  - Interior nodes remain free (unknowns to solve for)

### 3.4 Galerkin Method Structure

In the Galerkin finite element method:
- **Trial functions** ($u \in U$): Satisfy the boundary conditions and represent candidate solutions
- **Test functions** ($v \in V$): Satisfy homogeneous boundary conditions ($v = 0$ on $\partial\Omega$) and are used to form the weak formulation

The distinction ensures that when we test the residual with functions from $V$, the boundary conditions are properly enforced through the trial space $U$.

In [3]:
# Solve the Poisson problem
reffe = ReferenceFE(lagrangian,Float64,1)
V = FESpace(model,reffe;conformity=:H1,dirichlet_tags="boundary")
U = TrialFESpace(V,x->x[1])

UndefVarError: UndefVarError: `ReferenceFE` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 4. Defining Triangulation and Integration Measures

Now that we have defined our finite element spaces, we need to set up the infrastructure for performing numerical integration over the domain. This is essential for computing the integrals that appear in the weak formulation of our PDE.

### 4.1 Triangulation

```julia
Ω = Triangulation(model)
```

The `Triangulation` represents the computational domain decomposed into cells (elements):

- **`Ω`** (capital Omega): Denotes the computational domain
- Creates a triangulation object from our discrete model
- Provides the geometric information needed for integration:
  - Cell connectivity
  - Node coordinates
  - Jacobian transformations between reference and physical  (information required to do this operation)
  - ...
- In our case with `n=2`, we have 4 quadrilateral cells

### 4.2 Integration Measure

```julia
dΩ = Measure(Ω, 2)
```

The `Measure` defines how we perform numerical integration over the domain:

- **`dΩ`**: Represents the differential volume element (in mathematical notation: $d\Omega$)
- **`Ω`**: The triangulation over which we integrate
- **`2`**: The quadrature order (degree of precision)
  - Determines how many integration points per cell
  - Order 2 is exact for polynomials up to degree 2
  - For bilinear basis functions (order 1), this is sufficient for the stiffness matrix

### 4.3 Numerical Integration in Gridap

When we write integrals like $\int_\Omega \nabla u \cdot \nabla v \, d\Omega$ in Gridap, the `dΩ` measure handles:

1. **Quadrature rules**: Selects integration points and weights within each cell
2. **Information of the integration domain** in which different opperations are performed, e.g.:
  - **Coordinate transformation**: Maps from reference element to physical cells
  - **Jacobian computation**: Accounts for the volume change in the mapping
  - **Assembly**: Accumulates contributions from all cells

The beauty of Gridap's syntax is that `dΩ` makes the code look like the mathematical weak formulation!

> **📘 Note**: The quadrature order should be chosen based on the polynomial degree of the integrand. For linear elements with gradient operators, order 2 is typically sufficient. Higher-order elements would require higher quadrature orders.

In [4]:
Ω = Triangulation(model)
dΩ = Measure(Ω,2)

UndefVarError: UndefVarError: `Triangulation` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 5. Weak Form Definition and Solution

With all the components in place, we can now define the weak formulation of our Poisson equation and solve the resulting linear system.

### 5.1 Source Term

```julia
f(x) = 1.0
```

This defines the source term (right-hand side) of our Poisson equation $-\Delta u = f$:
- Constant source term $f = 1.0$ throughout the domain
- More complex functions could depend on position: `f(x) = sin(π*x[1])*cos(π*x[2])`

### 5.2 Bilinear Form (Left-Hand Side)

```julia
a(u,v) = ∫( ∇(v)⋅∇(u) )dΩ
```

The bilinear form `a(u,v)` represents the left-hand side of the weak formulation:

$$a(u,v) = \int_\Omega \nabla v \cdot \nabla u \, d\Omega$$

- **`∇(u)`** and **`∇(v)`**: Compute gradients of trial and test functions
- **`⋅`**: Dot product of the gradient vectors
- **`∫(...)dΩ`**: Integration over the domain using the measure we defined
- This corresponds to the weak form of $-\Delta u$ after integration by parts

### 5.3 Linear Form (Right-Hand Side)

```julia
l(v) = ∫( f*v )dΩ
```

The linear form `l(v)` represents the right-hand side:

$$l(v) = \int_\Omega f \, v \, d\Omega$$

- Multiplies the source term `f` by the test function `v`
- Integrated over the entire domain

### 5.4 Creating the FE Operator

```julia
op = AffineFEOperator(a, l, U, V)
```

The `AffineFEOperator` assembles the discrete system:
- Takes the bilinear form `a` and linear form `l`
- Uses trial space `U` and test space `V`
- Constructs the linear system: $\mathbf{A}\mathbf{u} = \mathbf{b}$
  - $\mathbf{A}_{ij} = a(\phi_j, \psi_i)$: Stiffness matrix
  - $\mathbf{b}_i = l(\psi_i)$: Load vector
- Applies boundary conditions from the trial space `U`

### 5.5 Solving the System

```julia
uh = solve(op)
```

The `solve` function:
- Solves the linear system $\mathbf{A}\mathbf{u} = \mathbf{b}$
- Returns a finite element function `uh` (the discrete solution)
- Uses appropriate linear solver (typically direct solver for small problems, iterative for large ones)

### 5.6 Visualization

```julia
writevtk(Ω, "solution", cellfields=["uh"=>uh])
```

Exports the solution to VTK format:
- Creates `solution.vtu` file
- Associates the solution field `uh` with the name "uh"
- Can be opened in ParaView to visualize the solution field

> **📘 Note**: Gridap's syntax closely mirrors mathematical notation, making it intuitive to translate weak formulations directly into code. The `∫` symbol can be typed with `\int<TAB>` in Julia.

In [5]:
f(x) = 1.0
a(u,v) = ∫( ∇(v)⋅∇(u) )dΩ
l(v) = ∫( f*v )dΩ
op = AffineFEOperator(a,l,U,V)
uh = solve(op)  
writevtk(Ω,"solution",cellfields=["uh"=>uh])

UndefVarError: UndefVarError: `AffineFEOperator` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 6. Inspecting the Assembled System Matrices

After solving the problem, it's instructive to examine the assembled linear system to understand what Gridap has constructed under the hood.

### 6.1 Extracting System Components

```julia
A = get_matrix(op)
b = get_vector(op)
```

These functions extract the discrete linear system from the FE operator:

- **`A`**: The global stiffness matrix (sparse format)
  - Size: $n_{dof} \times n_{dof}$ where $n_{dof}$ is the number of degrees of freedom
  - Symmetric positive definite for the Poisson problem
  - Most entries are zero (sparse structure) due to local support of basis functions
  
- **`b`**: The global load vector
  - Size: $n_{dof} \times 1$
  - Contains contributions from the source term and boundary conditions

### 6.2 Displaying the System

```julia
display(Matrix(A))
display(Vector(b))
```

Converting to dense formats for inspection:

- **`Matrix(A)`**: Converts sparse matrix to dense format for visualization
  - Shows the full matrix including zeros
  - Reveals the sparsity pattern and connectivity
  - For our $2 \times 2$ mesh, we have 9 nodes but some are constrained by boundary conditions
  
- **`Vector(b)`**: Displays the load vector as a standard array

### 6.3 What to Look For

When examining the assembled system:

1. **Matrix sparsity**: Non-zero entries only where basis functions overlap
2. **Symmetry**: For self-adjoint problems like Poisson, $A$ should be symmetric
3. **Diagonal dominance**: Often present in well-posed elliptic problems
4. **Boundary conditions**: Reflected in rows/columns corresponding to boundary nodes
5. **Load vector**: Should reflect the integrated source term over elements

> **📘 Note**: For large problems, converting to dense format is impractical. In production code, sparse matrices should be kept in sparse format. Here we convert for educational purposes only.

In [6]:
using SparseArrays
A = get_matrix(op)
b = get_vector(op)
display(Matrix(A))
display(Vector(b))


UndefVarError: UndefVarError: `get_matrix` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Understanding the System Size

The system is **1×1** because we only have **1 free degree of freedom**.

With a $2 \times 2$ mesh, we have $(n+1) \times (n+1) = 3 \times 3 = 9$ total nodes. However, boundary conditions are applied to all boundary nodes:

- **Boundary nodes**: 1, 2, 3, 4, 6, 7, 8, 9 (8 nodes total)
- **Interior node**: 5 (the center node)

This leaves only **node 5** as a free degree of freedom, which is why our system matrix is $1 \times 1$.

Let's verify this:

In [7]:
num_total_nodes = (n+1)^2
num_free_dofs = num_free_dofs(U)
num_fixed_dofs = num_total_nodes - num_free_dofs

println("Total nodes: $num_total_nodes")
println("Free degrees of freedom: $num_free_dofs")
println("Fixed degrees of freedom (boundary): $num_fixed_dofs")
println("\nTherefore, the system matrix A is $(num_free_dofs)×$(num_free_dofs)")

UndefVarError: UndefVarError: `n` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 7. Understanding Basis Functions: Reference vs Physical Domain

A fundamental concept in finite element methods is the distinction between the **reference element** and **physical elements**. Let's explore how basis functions are defined and evaluated in both domains.

### 7.1 Getting Basis Functions

```julia
u_basis = get_trial_fe_basis(U)  # Trial space basis
v_basis = get_fe_basis(V)        # Test space basis
```

These extract the basis functions from our finite element spaces:
- **Trial basis** (`u_basis`): The basis functions for the solution space
- **Test basis** (`v_basis`): The basis functions for testing (typically the same structure but with homogeneous boundary conditions)

### 7.2 Reference Element Basis Functions

For bilinear quadrilateral elements, the basis functions on the reference element $\hat{\Omega} = [0,1] \times [0,1]$ are:

$$\begin{align}
N_1(\xi_1, \xi_2) &= \xi_1 \xi_2 \\
N_2(\xi_1, \xi_2) &= \xi_1 (1-\xi_2) \\
N_3(\xi_1, \xi_2) &= (1-\xi_1) \xi_2 \\
N_4(\xi_1, \xi_2) &= (1-\xi_1)(1-\xi_2)
\end{align}$$

These are defined in the code as:
```julia
N1(x) = x[1]*x[2]
N2(x) = x[1]*(1-x[2])
N3(x) = (1-x[1])*x[2]
N4(x) = (1-x[1])*(1-x[2])
```

**Key Properties**:
- Each $N_i$ equals 1 at node $i$ and 0 at the other three nodes
- They are bilinear functions (products of linear functions in each direction)
- The sum of all basis functions equals 1: $\sum_{i=1}^4 N_i = 1$

### 7.3 Mapping Between Domains

Physical elements are mapped from the reference element via:

$$\mathbf{x}(\boldsymbol{\xi}) = \sum_{i=1}^4 \mathbf{x}_i N_i(\boldsymbol{\xi})$$

where $\mathbf{x}_i$ are the physical node coordinates.

**Isoparametric Mapping Visualization:**

```
Reference Element [0,1]×[0,1]          Physical Element
       ξ₂                                    x₂
        ↑                                     ↑
        |                                     |
    N₃  4---------3  N₁                    x₄ •---------• x₃
        |         |              →            |         |
        |    ξ    |                           |    x    |
        |         |                           |         |
    N₄  1---------2  N₂                    x₁ •---------• x₂
        └────────→ ξ₁                         └────────→ x₁

          Node ordering:                   Mapping: x = Σ xᵢ Nᵢ(ξ)
          1: (0,0) → N₄                   
          2: (1,0) → N₂                    Jacobian: J = ∂x/∂ξ
          3: (1,1) → N₁                   
          4: (0,1) → N₃                    Integration: ∫_Ωₑ f dΩ = ∫_[0,1]² f(x(ξ)) |J| dξ
```

**Key aspects of the mapping:**

1. **Isoparametric**: Same basis functions $N_i$ are used for both geometry and field interpolation
2. **Bilinear transformation**: Straight edges map to straight edges, but can handle parallelograms
3. **Jacobian matrix**: $\mathbf{J} = \frac{\partial \mathbf{x}}{\partial \boldsymbol{\xi}}$ transforms derivatives and integration measures
4. **Determinant**: $|\mathbf{J}|$ scales the integration measure from reference to physical element

### 7.4 Evaluating Basis Functions

```julia
xp = Point(x1, x2)           # Point in physical cell
u_basis(xp)                   # Evaluates all basis functions at xp

xp_ref = Point(1/3, 1/3)     # Point in reference element
[h(xp_ref) for h in ref_FE_basis_funs]  # Manual evaluation on reference element
```

When we evaluate basis functions at a physical point:
1. Gridap determines which cell contains the point
2. Maps the point back to the reference element
3. Evaluates reference basis functions
4. Applies necessary transformations (for gradients, this includes the Jacobian)

> **📘 Note**: Understanding the reference element is crucial for implementing custom finite elements or debugging integration issues. Gridap handles these transformations automatically, but knowing what happens behind the scenes is valuable for advanced usage.

In [8]:
using Gridap.FESpaces
using Gridap.CellData
u_basis = get_trial_fe_basis(U) # basis functions of the trial space
u_basis_data = get_data(u_basis)
@show u_basis_data
N1(x) = x[1]*x[2]
N2(x) = x[1]*(1-x[2])
N3(x) = (1-x[1])*x[2]
N4(x) = (1-x[1])*(1-x[2])
ref_FE_basis_funs = [N1,N2,N3,N4]

x1 = L/2 + L/3
x2 = L/3

xp = Point(x1,x2) # Point in the physical cell
@show u_basis(xp)

xp_ref = Point(1/3,1/3) # Point in the reference cell
@show xp_ref
@show ref_FE_basis_xp = [h(xp_ref) for h in ref_FE_basis_funs]

ArgumentError: ArgumentError: Package Gridap not found in current path.
- Run `import Pkg; Pkg.add("Gridap")` to install the Gridap package.

Analyse the output of previous cell

In [9]:
v_basis = get_fe_basis(V) # basis functions of the test space
@show v_basis(xp)

UndefVarError: UndefVarError: `get_fe_basis` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 8. Finite Element Functions: Free vs Fixed Degrees of Freedom

In finite element analysis with boundary conditions, degrees of freedom are divided into two categories: **free** (unknowns to solve for) and **fixed** (prescribed by boundary conditions).

### 8.1 Creating a Zero Function

```julia
uhd = zero(U)
```

This creates a finite element function in the trial space `U` with:
- All free degrees of freedom set to zero
- Boundary (Dirichlet) degrees of freedom set according to the boundary condition function
- In our case, boundary values are $u = x_1$ as specified when creating `U`

### 8.2 Understanding the Data Structure

```julia
uhd_data = get_data(uhd)
```

The `get_data` function reveals the internal representation:

- **Free DoFs**: Interior nodes where the solution is unknown (to be computed)
- **Fixed DoFs**: Boundary nodes where values are prescribed
- The data structure separates these to enable efficient assembly and solution

### 8.3 Degrees of Freedom Layout

For our $2 \times 2$ mesh with 9 total nodes:

```
Node layout:
7 --- 8 --- 9
|     |     |
4 --- 5 --- 6
|     |     |
1 --- 2 --- 3
```

- **Boundary nodes** (1, 2, 3, 4, 6, 7, 8, 9): Fixed by Dirichlet condition $u = x_1$
- **Interior node** (5): Free degree of freedom (the unknown we solve for)

### 8.4 Why This Matters for Assembly

When assembling the global system:

1. **Free-free contributions**: Go into the system matrix $\mathbf{A}$
2. **Free-fixed contributions**: Move to the right-hand side (known values from BCs)
3. **Fixed-fixed contributions**: Not assembled (BCs directly enforced)

This separation enables:
- Smaller system to solve (only free DoFs)
- Exact satisfaction of Dirichlet boundary conditions
- Efficient handling of constraints

### 8.5 Representing a Solution

A finite element function $u_h$ can be written as:

$$u_h(\mathbf{x}) = \sum_{i \in \text{free}} u_i \phi_i(\mathbf{x}) + \sum_{j \in \text{fixed}} u_j^{BC} \phi_j(\mathbf{x})$$

where:
- $u_i$ are the unknowns (free DoFs)
- $u_j^{BC}$ are prescribed boundary values (fixed DoFs)
- $\phi_i, \phi_j$ are the corresponding basis functions

> **📘 Note**: This separation of free and fixed DoFs is fundamental to efficiently imposing essential (Dirichlet) boundary conditions in the finite element method.

In [10]:
uhd = zero(U)
uhd_data = get_data(uhd)
@show uhd_data

# u_value_p1 = [1.0, 2.0, 3.0, 4.0]' * ref_FE_basis_xp

UndefVarError: UndefVarError: `U` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 9. Evaluating Weak Forms with Basis Functions

Now we dive into the assembly process by evaluating our weak forms using the basis functions directly. This reveals what happens under the hood when Gridap assembles the system matrices.

### 9.1 Computing Contributions

```julia
matcontrib = a(u_basis, v_basis)  # Matrix contributions
veccontrib = l(v_basis)            # Vector contributions
```

These evaluate the bilinear and linear forms with basis functions:

**Matrix contributions** (`matcontrib`):
- Evaluates $a(\phi_j, \psi_i)$ for all pairs of trial ($\phi_j$) and test ($\psi_i$) basis functions
- Results in element stiffness matrices for each cell
- For our problem: $a(\phi_j, \psi_i) = \int_{\Omega_e} \nabla \psi_i \cdot \nabla \phi_j \, d\Omega$

**Vector contributions** (`veccontrib`):
- Evaluates $l(\psi_i)$ for all test basis functions
- Results in element load vectors for each cell  
- For our problem: $l(\psi_i) = \int_{\Omega_e} f \, \psi_i \, d\Omega$

### 9.2 The Assembly Process

The contributions are computed **element-by-element**:

1. **Loop over cells**: For each element $\Omega_e$ in the mesh
2. **Local integration**: Compute element matrix $\mathbf{K}^e$ and vector $\mathbf{f}^e$
3. **Assembly mapping**: Determine which global DoFs correspond to local element nodes
4. **Accumulation**: Add element contributions to global system

### 9.3 Collecting Cell Data

```julia
data = collect_cell_matrix_and_vector(U, V, matcontrib, veccontrib, uhd)
```

This function collects the element-wise data:
- **Cell matrices**: Local stiffness matrices $\mathbf{K}^e$ for each element
- **Cell vectors**: Local load vectors $\mathbf{f}^e$ for each element
- **Assembly indices**: Mappings from local to global DoF numbering
- **Boundary data**: Contributions from fixed DoFs (from `uhd`)

### 9.4 Why This is Important

Understanding the element-by-element assembly reveals:
- How local computations on simple reference elements build the global system
- The source of matrix sparsity (local support of basis functions)
- How boundary conditions affect the assembly
- Where computational cost comes from in FEM

This level of detail is typically hidden in high-level FEM codes, but Gridap exposes it for users who want to understand or customize the assembly process.

> **📘 Note**: The next code cell will examine the actual numerical values of these element contributions, showing the concrete matrices and vectors computed during assembly.

In [11]:
matcontrib = a(u_basis,v_basis)
veccontrib = l(v_basis)

UndefVarError: UndefVarError: `u_basis` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [12]:
data = collect_cell_matrix_and_vector(U,V,matcontrib,veccontrib,uhd)

UndefVarError: UndefVarError: `collect_cell_matrix_and_vector` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
icell = 3
matvec_data = data[1]
celldata = matvec_data[1] # w
rows = matvec_data[2]     # r
cols = matvec_data[3]     # c

println("Cell data: ")
display(length(celldata[1]))
println("Rows: ")
display(rows)
println("Cols: ")
display(cols)

elemental_matrix_icell = Matrix(celldata[1][icell][1])
elemental_vector_icell = Vector(celldata[1][icell][2])
display(elemental_matrix_icell)
display(elemental_vector_icell)

UndefVarError: UndefVarError: `data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.